In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import pandas as pd


In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
train = pd.read_csv('iris_training.csv',names = CSV_COLUMN_NAMES,header=0)
test = pd.read_csv('iris_test.csv',names = CSV_COLUMN_NAMES,header=0)
train.head()
train_y = train.pop('Species')
test_y = test.pop('Species')

In [3]:
def input_fn(features,labels,training=True,batch_size=256):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)


In [4]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [7]:
#model creation
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[30,10],
    n_classes=3
)

classifier.train(input_fn = lambda: input_fn(train,train_y,training = True), steps = 100000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\cttpa\\AppData\\Local\\Temp\\tmpi7myjv6f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorf

In [8]:
eval_result = classifier.evaluate(input_fn = lambda: input_fn(test,test_y, training = False))
print('\nTest accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-26T14:00:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\cttpa\AppData\Local\Temp\tmpi7myjv6f\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.31207s
INFO:tensorflow:Finished evaluation at 2021-10-26-14:00:23
INFO:tensorflow:Saving dict for global step 100000: accuracy = 0.96666664, average_loss = 0.060704283, global_step = 100000, loss = 0.060704283
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100000: C:\Users\cttpa\AppData\Local\Temp\tmpi7myjv6f\model.ckpt-100000

Test accuracy: 0.967



In [11]:
def input_fn(features,batch_size = 256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features = ['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict = {}

print("Numerical Values as prompted")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn = lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100*probability))



Numerical Values as prompted
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\cttpa\AppData\Local\Temp\tmpi7myjv6f\model.ckpt-100000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (100.0%)
